In [4]:
!brew install tesseract

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:e02b387d80f10c835df15115360b0b5deb8e35f8967c7e68c9942af046023209
######################################################################### 100.0%
==> Pouring portable-ruby-3.3.5.el_capitan.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 5 taps (aquasecurity/trivy, hashicorp/tap, anchore/grype, homebrew/core and homebrew/cask).
==> New Formulae
action-docs                keep-sorted                probe-rs-tools
afl++                      kty                        python-freethreading
aicommit                   kubetail                   python-gdbm@3.13
ansible-builder            lbfgspp                    python-tk@3.13
argtable3                  libcss                     python@3.13
binsider                 

In [13]:
from openai import OpenAI
import pytesseract
from pdf2image import convert_from_path
import os
import base64

# Step 1: Extract text from each page in the PDF
def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    extracted_text = ""

    for page in images:
        text = pytesseract.image_to_string(page)
        extracted_text += text + "\n\n"
    
    return extracted_text

# Define PDF path and question
pdf_path = "FileADV.pdf"

# Extract text from the PDF and query the model
text_content = extract_text_from_pdf(pdf_path)



In [40]:
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

api_key = os.getenv("OAI")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

# System message for Part 1 with hard-coded content from the "Type of Client" table
system_message = """
You are an assistant trained to identify specific item numbers, question numbers, and sub-items from the Form ADV document to support SQL query generation. Your task is to locate the relevant columns with information necessary to answer user questions, which will later be manipulated with SQL in Part 2.

Important Information:
1. **Item 5.D - Type of Client Data**:
   - 5D(a)(1) to 5D(n)(1): Number of Clients for each client type, structured as follows:
     - 5D(a)(1): "Individuals (other than high net worth individuals)"
     - 5D(b)(1): "High net worth individuals"
     - 5D(c)(1): "Banking or thrift institutions"
     - (continues similarly through each client type up to)
     - 5D(n)(1): "Other"
   - Each client type (from a to n) also has:
     - 5D(a)(2) to 5D(n)(2): Column indicating if there are fewer than 5 clients for each type.
     - 5D(a)(3) to 5D(n)(3): Amount of Regulatory Assets under Management for each client type.


2. **Item 9 - Custody of Client Assets**:
   - 9A(1)(a): Custody of "cash or bank accounts" by the firm itself.
   - 9A(1)(b): Custody of "securities" by the firm itself.
   - 9A(2)(a): Approximate amount of client funds in custody by the firm.
   - 9A(2)(b): Total number of clients for which the firm has custody of assets.
   - 9B(1)(a) to 9B(2)(b): Similar fields for custody by related persons.

3. **Definition of "Small Registered Investment Advisors"**:
  - An advisor is considered "small" if they answered "No" to all the following items in Form ADV:
    - Item 12A
    - Item 12B(1)
    - Item 12B(2)
    - Item 12C(1)
    - Item 12C(2)
  - This is used as the small entity definition for identifying "small registered investment advisors" (RIAs).

4. **Cell Values for Yes/No Answers**:
   - Responses are typically represented by "Y" for Yes and "N" for No in the columns. When identifying relevant columns based on a question, ensure that you check if the presence of a "Y" or "N" response affects whether the column should be included.
   - Example: If the question asks to find advisors that are "small," include columns 12A, 12B1, 12B2, 12C1, and 12C2, and indicate that an "N" in each column signifies a small advisor.


**Instructions**:

1. **Find Relevant Columns**: Your primary goal is to locate the columns containing data necessary to calculate the answer, even if the form does not contain an exact match for the question.
   - Example: If asked for the fraction of advisors with fewer than 2000 clients, identify columns that record client counts (Item 5.D (e.g., 5D(a)(1), 5D(b)(1), etc.)) as relevant data sources.

2. **Interpreting Specific and Threshold-Based Questions**:
   - When a question includes thresholds or specific counts (e.g., "more than one million clients"), identify the columns with general client counts as relevant data.
   - If the question involves categories without precise values (e.g., "fraction of advisors with a certain trait"), include all relevant columns even if they do not provide an exact match for the threshold.

3. **Provide Closest Match**: If an exact match isn’t available, give the closest item and sub-item numbers relevant to the question's context.

4. **Be concise**: 
    - Provide the item numbers and sub-items that contain the most relevant data.
    - Avoid unnecessary information that does not directly help answer the question.
    
**Examples**:
- Question: "What is the total number of assets under management of the investment advisers, in trillion dollars?"
  - Answer: Item 5.D, specifically columns 5D(a)(3), 5D(b)(3), ..., etc. for all client types.

- Question: "What is the number of advisers with each more than one million clients?"
  - Answer: Item 5.D, specifically columns 5D(a)(1), 5D(b)(1), ..., 5D(c)(1), etc.

- Question: "What fraction of advisers provide portfolio management services to their clients?"
  - Answer: The relevant information is in 5.G.(2), 5.G.(3), 5.G.(4), and 5.G.(5).

- Question: "What is the total assets under custody of advisers, in trillion dollars?"
  - Answer: Item 9.A(2)(a) for the firm's custody of client funds and Item 9.B(2)(a) for related persons' custody of client funds.

Focus on providing item numbers and sub-items based on the closest information relevant to the question's requirements.

"""

# Step 2: Query OpenAI model with extracted text
def query_openai_gpt(text, question):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": f"{question}\n\nDocument Text:\n{text}"
            }
        ]
    )
    return completion.choices[0].message

question = """What fraction of advisers provide portfolio management services to their clients?"""

response = query_openai_gpt(text_content, question)
print(response)

question = """What is the number of advisers with each more than one million clients?"""

response = query_openai_gpt(text_content, question)
print(response)

question = """What fraction of advisors have fewer than 2000 clients?"""

response = query_openai_gpt(text_content, question)
print(response)

question = """What is the total number of small registered investment advisors?"""

response = query_openai_gpt(text_content, question)
print(response)

ChatCompletionMessage(content='To determine the fraction of advisers providing portfolio management services to their clients, you should refer to the advisory activities section, specifically:\n\n- Item 5.G, which lists the types of advisory services provided. Relevant sub-items include:\n  - 5.G.(2) Portfolio management for individuals and/or small businesses\n  - 5.G.(3) Portfolio management for investment companies\n  - 5.G.(4) Portfolio management for pooled investment vehicles\n  - 5.G.(5) Portfolio management for businesses or institutional clients\n\nThese columns will provide the necessary information to identify advisers offering portfolio management services.', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='To answer the question "What is the number of advisers with each more than one million clients?" we would need to reference data that tracks client counts for investment advisers. The relevant item and sub-items in Form

In [41]:
question = """What is the fraction of advisers having custody of clients' cash or securities?"""

response = query_openai_gpt(text_content, question)
print(response)

ChatCompletionMessage(content='To determine the fraction of advisers having custody of clients\' cash or securities, you need to examine the following items:\n\n- Item 9.A(1)(a): Custody of "cash or bank accounts" by the firm itself.\n- Item 9.A(1)(b): Custody of "securities" by the firm itself.\n- Item 9.B(1)(a): Custody of "cash or bank accounts" by related persons.\n- Item 9.B(1)(b): Custody of "securities" by related persons.\n\nThese items will provide the necessary information to identify advisers with custody over clients\' assets.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [42]:
question = "Which question asks if you have ever been convicted of a felony? Please respond with the exact item number, question number, and letter if it exists."

response = query_openai_gpt(text_content, question)
print(response)

ChatCompletionMessage(content='The question about whether you have ever been convicted of a felony is located in Item 11.A.(1) of the Form ADV.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [43]:
question = """Which question(s) ask for the number of clients a firm has that are both 
individuals and individuals of high net worth? Please respond with the exact item number(s), 
question number(s), and letter(s)."""

response = query_openai_gpt(text_content, question)
print(response)

ChatCompletionMessage(content='The questions or item numbers that ask for the number of clients a firm has that are both individuals and individuals of high net worth can be found under:\n\n- Item 5.D, specifically:\n  - 5D(a)(1): "Individuals (other than high net worth individuals)"\n  - 5D(b)(1): "High net worth individuals"\n\nThese sub-items of Item 5.D are used to gather data on the number of clients for these categories.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [ ]:
import pandas as pd
import sqlite3

# Load the Excel file 
excel_data = pd.read_excel('RegisteredAdvisors.xlsx')

# Connect to SQLite Database (or create one if it doesn't exist)
conn = sqlite3.connect("RegisteredAdvisors.db")
excel_data.to_sql("RegisteredAdvisors", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [62]:
import openai

# Part 2 System Message for SQL Query Generation

all_columns = list(excel_data.columns)

part2_system_message = f"""
You are a SQL assistant with knowledge of the column structure of a financial advisors database.
Here is a list of column names:
{', '.join(map(str, all_columns))}

Mapping Examples:
- "Item 1.A" maps to "1A".
- "Item 5.B.(1)" maps to "5B(1)".
- "Item 6, Part A" maps to ["6A(1)", "6A(2)", ... "6A(14)", "6A(14)-Other"].

Instructions:
1. Use the answer from Phase 1, which contains an item identifier, to identify the corresponding column(s) in the database.
    - If the identifier maps to a single column, use that column.
    - If it maps to a group of columns, include all columns in that group.
    - If the answer is broad, such as "Item 9.A.(1)," and there are sub-columns like "9A(1)(a)" and "9A(1)(b)," include all relevant sub-columns in the query to capture all parts of the subquestion.
    - Some questions may directly map to specific columns based on their context, even without an identifier. Use column names directly if they align with the question, such as "Total Gross Assets of Private Funds."
2. Refer to the original question to determine the intent and any specific conditions for the SQL query:
    - If the question asks for an "average," use the AVG function.
    - If it requests a "total," use SUM.
    - If it asks for a count, use COUNT.
    - Use other SQL functions or aggregation as appropriate based on the context.
3. Combine the columns identified from the answer in Phase 1 with the intent and conditions inferred from the original question.
4. Structure the SQL output flexibly, based on the user's intent. Here are some example formats:
    - For direct selection: SELECT [columns] FROM RegisteredAdvisors WHERE [conditions if any].
    - For aggregation: SELECT AVG([column]) FROM RegisteredAdvisors WHERE [conditions if any] (if the question asks for averages).
    - Apply similar structures for SUM, COUNT, and any other relevant SQL functionality.
    - If the question is asking for a specific company, include a WHERE clause to filter by the company name. In the database, all company names are in all caps in the 'Primary Business Name' column.
5. Use the appropriate SQL format based on the type of information requested and return the generated SQL query.

Additional Information: 
1. **Legal Name Formatting**: 
  - The legal name of a business is in all caps in the data. If the question is about a specific firm, 
  ensure that you use the legal name in all caps for accurate identification. 
  - Ex. If the question asks for the total assets under management of "ABC Financial Services," 
    you would use the legal name "ABC FINANCIAL SERVICES" in the where clause of the query.
    
Example Scenarios:
- Question: "What is the fraction of advisers having custody of clients' cash or securities?"
  - Part 1 Answer: Relevant columns are 9A(1)(a) and 9A(1)(b).
  - SQL Query: `SELECT COUNT(*) * 1.0 / (SELECT COUNT(*) FROM RegisteredAdvisors.db) AS fraction_having_custody FROM RegisteredAdvisors WHERE "9A(1)(a)" = 'Y' OR "9A(1)(b)" = 'Y';`

- Question: "What is the total number of assets under management of the investment advisers, in trillion dollars?"
  - Part 1 Answer: Columns 5D(a)(3), 5D(b)(3), ..., to 5D(n)(3).
  - SQL Query: `SELECT SUM("5D(a)(3)" + "5D(b)(3)" + ... + 5D(n)(3)) / 1e12 AS total_assets_in_trillions FROM RegisteredAdvisors;`

Respond with ONLY the full SQL query based on the information provided in Part 1 and the question's 
intent. Don't include anything else in the response besides the exact SQL query so the entire answer
can be directly passed to the SQL interpreter without any need for cleaning.
"""

  
# Function to generate SQL query
def generate_sql_query(question, part1_answer):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": part2_system_message},
            {"role": "user", "content": f"Original Question: {question}\n\nPart 1 Answer: {part1_answer}"}
        ]
    )
    return completion.choices[0].message.content

# Example usage
question = "What is the number of small registered investment advisors?"
part1_answer = "Relevant columns are 12A, 12B1, 12B2, 12C1, and 12C2, with a 'N' response in each indicating a small advisor."

# Generate SQL query
sql_query = generate_sql_query(question, part1_answer)
print(sql_query)

```sql
SELECT COUNT(*) AS number_of_small_advisors
FROM RegisteredAdvisors
WHERE "12A" = 'N' AND "12B(1)" = 'N' AND "12B(2)" = 'N' AND "12C(1)" = 'N' AND "12C(2)" = 'N';
```


Part 3: Executing SQL Query

1. Execute the SQL query on the database 
2. Interpret the Result and form a clear, detailed answer based on both the SQL output and the original question asked.

In [2]:
sql_query = """
```sql
SELECT COUNT(*) AS number_of_small_advisors
FROM RegisteredAdvisors
WHERE "12A" = 'N' AND "12B(1)" = 'N' AND "12B(2)" = 'N' AND "12C(1)" = 'N' AND "12C(2)" = 'N';
```
"""
# Clean sql query
sql_query = sql_query.replace("```sql", "").replace("```", "").strip()


print(sql_query)



SELECT COUNT(*) AS number_of_small_advisors
FROM RegisteredAdvisors
WHERE "12A" = 'N' AND "12B(1)" = 'N' AND "12B(2)" = 'N' AND "12C(1)" = 'N' AND "12C(2)" = 'N';


In [3]:
import sqlite3


# Define a function to execute SQL queries
def execute_sql_query(query):
    connection = sqlite3.connect("RegisteredAdvisors.db")
    cursor = connection.cursor()
    
    try: 
        cursor.execute(query)
        
        result = cursor.fetchone()
        
        return result[0] if result else None
    except Exception as e:
        print(f"Error executing query: {e}")
        return None
    finally:
        connection.close()

sql_response = execute_sql_query(sql_query)
print(sql_response)

591


In [8]:

from dotenv import load_dotenv
import os
from openai import OpenAI


# Load environment variables from the .env file
load_dotenv()

api_key = os.getenv("OAI")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

def get_final_answer_from_llm(question, sql_result):
    # Compose the prompt for the final answer generation
    prompt = f"""
    Here is a question asked by the user: "{question}"
    The result of the SQL query for this question is: {sql_result}
    
    Based on this result, provide a clear and detailed answer to the user, making sure to interpret the result in the context of the original question.
    """
    
    # Query OpenAI GPT with the composed prompt
    completion = client.chat.completions.create(
        model="gpt-4",  # or "gpt-4-turbo" or another model as needed
        messages=[
            {"role": "system", "content": "You are an assistant skilled at interpreting SQL query results."},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Extract and return the assistant's response
    return completion.choices[0].message.content

question = "What is the number of small registered investment advisors?"
final_answer = get_final_answer_from_llm(question, sql_response)
print(final_answer)

Based on the result of the SQL query, the number of small registered investment advisors is 591.
